## Requirements Relationships Chord Diagram


In [1]:
import re
from itertools import combinations
import psycopg2
import numpy as np
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from IPython import display
from bs4 import BeautifulSoup as bs

from nbstyler import DATA_STYLE as style

plotly.offline.init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline

%matplotlib notebook
%matplotlib inline

### Data Preparation

In [2]:
data_querystr = """SELECT * FROM v_full_data_offers_history"""
conn = psycopg2.connect('dbname=jobsbg')
data_df = pd.read_sql_query(data_querystr, conn, index_col='subm_date')
conn.close()

In [3]:
data_df.head(1)

,subm_type,job_id,company_id,norm_salary,job_title,company_name,text_salary,job_contents
subm_date,,,,,,,,
2017-09-27,submission,3994437,124912,NaN,Data Analyst,ПрайсуотърхаусКупърс Одит ООД,None,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 ..."


### Relationship Categories Definition

In order to build the necessary data we first need to define the scope of the relationships we aim to observe. The search strings we will compare should be of the same semantic category to provide meaningful insight.

To explore the data and prepare proper definitions we took a deep dive in the data offers' text contents. We used the `nltk` library for text processing to produce frequency distributions of most common words, bigrams and trigrams. Then those distributions were used to explore different possible relationship categories. See the details in the [Requirements Deep Dive Notebook](./Data_Offers_Requirements_Deep_Dive.ipynb).

For the first chord diagram the chosen view is a breakdown by technology. Other possible relationships to explore are skill requirements breakdown, perks & benefits breakdown, job title breakdown, etc.

### Data Jobs Technology Requirement Relationships

A preliminary list of terms that describe most often sought categories is presented below. We are going to use it to construct regex filters for selected key technologies.

In [4]:
#tech_terms_filters = [
#    r'excel', r'tableau', r'qlik', r'hadoop', r'ss[ir]s', r'sql server', r'power bi', 
#    r'spark', r'postgresql', r'informatica', r'microstrategy', r'(mysql)|(mariadb)']
#
#tech_terms_labels = [
#    'Excel', 'Tableau', 'Qlik', 'Hadoop', 'SSRS/SSIS', 'SQL Server', 'Power BI', 
#    'Spark', 'Postgresql', 'Informatica', 'Microstrategy', 'MySQL/MariaDB']

tech_terms_filters = [
    r'power bi', r'excel', r'ss[ir]s',
    r'sql server', r'postgresql', r'(mysql)|(mariadb)', r't.?sql', r'pl.?sql',
    r'pentaho', r'hadoop', r'spark', r'informatica',
    r'qlik', r'tableau', r'microstrategy', r'oracle (bi)|(business intelligence)',
    r'python', r'vba', r'linux', r'aws']

tech_terms_labels = [
    'Power BI', 'Excel', 'SSIS/SSRS',
    'MS SQL Server', 'PostgreSQL', 'MySQL/MariaDB', 'T-SQL', 'PL/SQL',
    'Pentaho', 'Hadoop', 'Spark', 'Informatica', 
    'Qlik', 'Tableau', 'Microstrategy', 'Oracle BI',
    'Python', 'VBA', 'Linux', 'AWS']

#### Preparing a square matrix with counts for matching filters

First we define a helper function that returns `True` for job offers where both of the provided patterns are found in the job contents. We also prepare a list of all possible filter patterns' combinations.

In [5]:
def match_terms(first_term, second_term, text):
    if re.search(first_term, text, re.IGNORECASE) and re.search(second_term, text, re.IGNORECASE):
        return True
    else:
        return False
    
def count_matches(first_term, second_term, col):
    return sum([match_terms(first_term, second_term, t) for t in col.values])

req_combinations = list(combinations(tech_terms_filters, 2))
req_combinations[:5]

[('power bi', 'excel'),
 ('power bi', 'ss[ir]s'),
 ('power bi', 'sql server'),
 ('power bi', 'postgresql'),
 ('power bi', '(mysql)|(mariadb)')]

Building the counts in a dictionary with keys composed of tuples with both search terms. 

In [6]:
%%time
match_results = [count_matches(*tup, data_df.job_contents) for tup in req_combinations]
match_dict = dict(zip(req_combinations, match_results))

CPU times: user 1min 20s, sys: 48.6 ms, total: 1min 20s
Wall time: 1min 20s


Another helper function will unpack the combinations counts into a square matrix form. Finally, a DataFrame is created from the combinations counts matrix. This is our main data source for the chord diagram. 

In [7]:
def make_matrix(headers, counts):
    res = []
    for k1 in headers:
        row = []
        for k2 in headers:
            if k1 == k2:
                row.append(0)
            else:
                curr_key = tuple([k1, k2])
                cell_value = counts.get(curr_key) if curr_key in counts else counts.get(tuple([k2, k1]))
                row.append(cell_value)
        res.append(row)
                
    return np.array(res, dtype=int)    

In [8]:
tech_terms_matrix = make_matrix(tech_terms_filters, match_dict)
tech_terms_matrix

array([[  0, 136,  78,  64,   0,   5,  28,   3,   7,  14,  19,   5,  89,
         89,   9,  63,  33,  20,   0,   3],
       [136,   0, 300, 159,  25,  64, 111,  36,  42,  50,  55,  79, 237,
        250,  23, 348, 189, 145,  50,  24],
       [ 78, 300,   0, 142,  11,  28,  91,  18,  18,  26,  35,  43,  43,
         82,  12, 139,  97,  47,  14,  14],
       [ 64, 159, 142,   0,  26,  33, 117,   6,  27,  20,  16,  32,  25,
         42,  10,  76,  53,  29,  26,   5],
       [  0,  25,  11,  26,   0,  35,  24,   0,  28,   3,   5,   0,   6,
          5,   0,  12,  12,   0,  20,   0],
       [  5,  64,  28,  33,  35,   0,  29,   0,  41,   3,  18,   1,  10,
         26,   0,  30,  29,   1,  27,   0],
       [ 28, 111,  91, 117,  24,  29,   0,  17,  26,  14,  14,   9,  38,
         52,   6,  68,  39,  21,  18,   6],
       [  3,  36,  18,   6,   0,   0,  17,   0,   6,   3,   0,  12,   8,
         11,  14,  53,   2,   5,   1,   0],
       [  7,  42,  18,  27,  28,  41,  26,   6,   0,   0,   6,  

In [9]:
tech_terms_df = pd.DataFrame(tech_terms_matrix, columns=tech_terms_labels, index=tech_terms_labels)
tech_terms_df

,Power BI,Excel,SSIS/SSRS,MS SQL Server,PostgreSQL,MySQL/MariaDB,T-SQL,PL/SQL,Pentaho,Hadoop,Spark,Informatica,Qlik,Tableau,Microstrategy,Oracle BI,Python,VBA,Linux,AWS
Power BI,0,136,78,64,0,5,28,3,7,14,19,5,89,89,9,63,33,20,0,3
Excel,136,0,300,159,25,64,111,36,42,50,55,79,237,250,23,348,189,145,50,24
SSIS/SSRS,78,300,0,142,11,28,91,18,18,26,35,43,43,82,12,139,97,47,14,14
MS SQL Server,64,159,142,0,26,33,117,6,27,20,16,32,25,42,10,76,53,29,26,5
PostgreSQL,0,25,11,26,0,35,24,0,28,3,5,0,6,5,0,12,12,0,20,0
MySQL/MariaDB,5,64,28,33,35,0,29,0,41,3,18,1,10,26,0,30,29,1,27,0
T-SQL,28,111,91,117,24,29,0,17,26,14,14,9,38,52,6,68,39,21,18,6
PL/SQL,3,36,18,6,0,0,17,0,6,3,0,12,8,11,14,53,2,5,1,0
Pentaho,7,42,18,27,28,41,26,6,0,0,6,20,16,23,2,25,23,1,25,11
Hadoop,14,50,26,20,3,3,14,3,0,0,77,4,8,29,3,23,85,6,49,19


We can finally move to Plotly. 

A chord diagram encodes information in two graphical objects:

- Ideograms, represented by distinctly colored arcs of circles;
- Ribbons, that are planar shapes bounded by two quadratic Bezier curves and two arcs of circle,that can degenerate to a point;


### Ideograms Preparation

For each of our predefined tech terms we can produce a total hits count by summing up all the entries on the row (or column for that matter). That total count determines the size of each ideogram of the chart.

We are going to need a couple of helper functions to process the data in order to get ideogram ends.


In [10]:
PI = np.pi

def moduloAB(x, a, b): #maps a real number onto the unit circle identified with 
                       #the interval [a,b), b-a=2*PI
        if a>=b:
            raise ValueError('Incorrect interval ends')
        y=(x-a)%(b-a)
        return y+b if y<0 else y+a

def test_2PI(x):
    return 0<= x <2*PI

And now use them to compute the row sums and the lengths of corresponding ideograms.

In [11]:
row_sum=[np.sum(tech_terms_matrix[k,:]) for k in range(len(tech_terms_filters))]

#set the gap between two consecutive ideograms
gap=2*PI*0.005
ideogram_length=2*PI*np.asarray(row_sum)/sum(row_sum)-gap*np.ones(len(tech_terms_filters))

The next function returns the list of end angular coordinates for each ideogram arc:


In [12]:
def get_ideogram_ends(ideogram_len, gap):
    ideo_ends=[]
    left=0
    for k in range(len(ideogram_len)):
        right=left+ideogram_len[k]
        ideo_ends.append([left, right])
        left=right+gap
    return ideo_ends

ideo_ends=get_ideogram_ends(ideogram_length, gap)
ideo_ends

[[0, 0.26069245079033426],
 [0.2921083773262322, 1.2810950982021194],
 [1.3125110247380174, 1.8248998668184735],
 [1.8563157933543715, 2.223748598566171],
 [2.2551645251020687, 2.316871870736338],
 [2.348287797272236, 2.483790943494185],
 [2.5152068700300827, 2.8035727460407966],
 [2.8349886725766944, 2.889228586008639],
 [2.920644512544537, 3.04165205507962],
 [3.0730679816155178, 3.23316972803336],
 [3.2645856545692578, 3.4413793082628845],
 [3.4727952347987823, 3.6324577204988406],
 [3.6638736470347384, 4.066886570387027],
 [4.0983024969229245, 4.613326903310083],
 [4.644742829845981, 4.691954571793385],
 [4.723370498329283, 5.362705687849259],
 [5.394121614385157, 5.800648623479716],
 [5.832064550015613, 5.99963372863578],
 [6.031049655171678, 6.154692762013465],
 [6.186108688549362, 6.251769380643686]]

The function make_ideogram_arc returns equally spaced points on an ideogram arc, expressed as complex numbers in polar form:

In [13]:
def make_ideogram_arc(R, phi, a=50):
    # R is the circle radius
    # phi is the list of ends angle coordinates of an arc
    # a is a parameter that controls the number of points to be evaluated on an arc
    if not test_2PI(phi[0]) or not test_2PI(phi[1]):
        phi=[moduloAB(t, 0, 2*PI) for t in phi]
    length=(phi[1]-phi[0])% 2*PI
    nr=5 if length<=PI/4 else int(a*length/PI)

    if phi[0] < phi[1]:
        theta=np.linspace(phi[0], phi[1], nr)
    else:
        phi=[moduloAB(t, -PI, PI) for t in phi]
        theta=np.linspace(phi[0], phi[1], nr)
    return R*np.exp(1j*theta)

In [14]:
z = make_ideogram_arc(1.3, [11*PI/6, PI/17])
z

array([1.12583302-0.65j      , 1.14814501-0.60972373j,
       1.16901672-0.5686826j , 1.18842197-0.5269281j ,
       1.20633642-0.48451259j, 1.22273759-0.44148929j,
       1.23760491-0.39791217j, 1.25091973-0.3538359j ,
       1.26266534-0.30931575j, 1.27282702-0.26440759j,
       1.28139202-0.21916775j, 1.28834958-0.17365297j,
       1.29369099-0.12792036j, 1.29740954-0.08202728j,
       1.29950058-0.0360313j , 1.29996146+0.01000988j,
       1.29879163+0.0560385j , 1.29599253+0.10199682j,
       1.2915677 +0.1478272j , 1.28552267+0.19347214j,
       1.27786503+0.23887437j])

### Ribbons Preparation

The function map_data maps all matrix entries to the corresponding values in the intervals associated to ideograms:

In [15]:
def map_data(data_matrix, row_value, ideogram_length):
    mapped=np.zeros(data_matrix.shape)
    for j  in range(len(tech_terms_filters)):
        mapped[:, j]=ideogram_length*data_matrix[:,j]/row_value
    return mapped

mapped_data=map_data(tech_terms_matrix, row_sum, ideogram_length)
mapped_data


array([[0.        , 0.05331455, 0.03057746, 0.0250892 , 0.        ,
        0.00196009, 0.01097652, 0.00117606, 0.00274413, 0.00548826,
        0.00744836, 0.00196009, 0.03488967, 0.03488967, 0.00352817,
        0.02469718, 0.01293662, 0.00784037, 0.        , 0.00117606],
       [0.05790021, 0.        , 0.12772106, 0.06769216, 0.01064342,
        0.02724716, 0.04725679, 0.01532653, 0.01788095, 0.02128684,
        0.02341553, 0.03363321, 0.10089964, 0.10643421, 0.00979195,
        0.14815643, 0.08046427, 0.06173184, 0.02128684, 0.01021768],
       [0.03228298, 0.12416531, 0.        , 0.05877158, 0.00455273,
        0.01158876, 0.03766348, 0.00744992, 0.00744992, 0.01076099,
        0.01448595, 0.01779703, 0.01779703, 0.03393852, 0.00496661,
        0.05752993, 0.04014678, 0.01945257, 0.00579438, 0.00579438],
       [0.02589835, 0.06434121, 0.05746196, 0.        , 0.0105212 ,
        0.01335384, 0.04734542, 0.00242797, 0.01092587, 0.00809323,
        0.00647459, 0.01294917, 0.01011654, 0

The array idx_sort, defined below, has on each row the indices that sort the corresponding row in mapped_data:


In [16]:
idx_sort=np.argsort(mapped_data, axis=1)
idx_sort

array([[ 0, 18,  4,  7, 19,  5, 11,  8, 14,  9, 10, 17,  6, 16, 15,  3,
         2, 12, 13,  1],
       [ 1, 14, 19,  4,  7,  8, 18,  9, 10,  5, 11,  6,  0, 17,  3, 16,
        12, 13,  2, 15],
       [ 2,  4, 14, 19, 18,  8,  7,  9,  5, 10, 11, 12, 17,  0, 13,  6,
        16, 15,  3,  1],
       [ 3, 19,  7, 14, 10,  9, 12,  4, 18,  8, 17, 11,  5, 13, 16,  0,
        15,  6,  2,  1],
       [ 0, 17, 14, 11,  7, 19,  4,  9, 10, 13, 12,  2, 15, 16, 18,  6,
         1,  3,  8,  5],
       [19, 14,  7,  5, 17, 11,  9,  0, 12, 10, 13, 18,  2, 16,  6, 15,
         3,  4,  8,  1],
       [ 6, 19, 14, 11, 10,  9,  7, 18, 17,  4,  8,  0,  5, 12, 16, 13,
        15,  2,  1,  3],
       [19, 10,  4,  5,  7, 18, 16,  0,  9, 17,  8,  3, 12, 13, 11, 14,
         6,  2,  1, 15],
       [ 9,  8, 17, 14, 10,  7,  0, 19, 12,  2, 11, 13, 16, 18, 15,  6,
         3,  4,  5,  1],
       [ 9,  8, 14,  4,  5,  7, 11, 17, 12,  0,  6, 19,  3, 15,  2, 13,
        18,  1, 10, 16],
       [ 7, 10, 14, 11,  4, 17

In [17]:
def make_ribbon_ends(mapped_data, ideo_ends,  idx_sort):
    L=mapped_data.shape[0]
    ribbon_boundary=np.zeros((L,L+1))
    for k in range(L):
        start=ideo_ends[k][0]
        ribbon_boundary[k][0]=start
        for j in range(1,L+1):
            J=idx_sort[k][j-1]
            ribbon_boundary[k][j]=start+mapped_data[k][J]
            start=ribbon_boundary[k][j]
    return [[(ribbon_boundary[k][j],ribbon_boundary[k][j+1] ) for j in range(L)] for k in range(L)]

ribbon_ends=make_ribbon_ends(mapped_data, ideo_ends,  idx_sort)
print('ribbon ends starting from the ideogram[2]\n', ribbon_ends[2])



ribbon ends starting from the ideogram[2]
 [(1.3125110247380174, 1.3125110247380174), (1.3125110247380174, 1.3170637527371167), (1.3170637527371167, 1.3220303650997707), (1.3220303650997707, 1.3278247461895336), (1.3278247461895336, 1.3336191272792965), (1.3336191272792965, 1.3410690458232772), (1.3410690458232772, 1.348518964367258), (1.348518964367258, 1.3592799578196746), (1.3592799578196746, 1.3708687199992002), (1.3708687199992002, 1.3853546727236072), (1.3853546727236072, 1.4031517003564502), (1.4031517003564502, 1.4209487279892932), (1.4209487279892932, 1.4404012930763541), (1.4404012930763541, 1.472684273433604), (1.472684273433604, 1.506622791245072), (1.506622791245072, 1.5442862683285306), (1.5442862683285306, 1.5844330515933158), (1.5844330515933158, 1.6419629781273897), (1.6419629781273897, 1.700734557752127), (1.700734557752127, 1.8248998668184733)]


In [18]:
def control_pts(angle, radius):
    #angle is a  3-list containing angular coordinates of the control points b0, b1, b2
    #radius is the distance from b1 to the  origin O(0,0) 

    if len(angle)!=3:
        raise InvalidInputError('angle must have len =3')
    b_cplx=np.array([np.exp(1j*angle[k]) for k in range(3)])
    b_cplx[1]=radius*b_cplx[1]
    return zip(b_cplx.real, b_cplx.imag)

In [19]:
def ctrl_rib_chords(l, r, radius):
    # this function returns a 2-list containing control poligons of the two quadratic Bezier
    #curves that are opposite sides in a ribbon
    #l (r) the list of angular variables of the ribbon arc ends defining 
    #the ribbon starting (ending) arc 
    # radius is a common parameter for both control polygons
    if len(l)!=2 or len(r)!=2:
        raise ValueError('the arc ends must be elements in a list of len 2')
    return [control_pts([l[j], (l[j]+r[j])/2, r[j]], radius) for j in range(2)]

In [20]:
def make_q_bezier(b):# defines the Plotly SVG path for a quadratic Bezier curve defined by the list of its control points
    if len(b)!=3:
        raise valueError('control poligon must have 3 points')
    A, B, C=b
    return 'M '+str(A[0])+',' +str(A[1])+' '+'Q '+\
                str(B[0])+', '+str(B[1])+ ' '+\
                str(C[0])+', '+str(C[1])

b=[(1,4), (-0.5, 2.35), (3.745, 1.47)]

make_q_bezier(b)


'M 1,4 Q -0.5, 2.35 3.745, 1.47'

In [21]:
def make_ribbon_arc(theta0, theta1):
    if test_2PI(theta0) and test_2PI(theta1):
        if theta0 < theta1:
            theta0= moduloAB(theta0, -PI, PI)
            theta1= moduloAB(theta1, -PI, PI)
            if theta0*theta1>0:
                raise ValueError('incorrect angle coordinates for ribbon')

        nr=int(40*(theta0-theta1)/PI)
        if nr<=2: nr=3
        theta=np.linspace(theta0, theta1, nr)
        pts=np.exp(1j*theta)# points on arc in polar complex form

        string_arc=''
        for k in range(len(theta)):
            string_arc+='L '+str(pts.real[k])+', '+str(pts.imag[k])+' '
        return   string_arc
    else:
        raise ValueError('the angle coordinates for an arc side of a ribbon must be in [0, 2*pi]')

In [22]:
def make_ideo_shape(path, line_color, fill_color):
    #line_color is the color of the shape boundary
    #fill_collor is the color assigned to an ideogram
    return dict(
        line=dict(color=line_color, width=0.45),
        path=path,
        type='path',
        fillcolor=fill_color,
        layer='below',)

In [23]:
temp_colors = [*style['colorramp']['acc1'], *style['colorramp']['acc2']]
len_datalist = len(tech_terms_filters)
ideo_colors=temp_colors[:len_datalist]

In [24]:
def make_ribbon(l, r, line_color, fill_color, radius=0.2):
    #l=[l[0], l[1]], r=[r[0], r[1]]  represent the opposite arcs in the ribbon 
    #line_color is the color of the shape boundary
    #fill_color is the fill color for the ribbon shape
    poligon=ctrl_rib_chords(l,r, radius)
    b,c =poligon

    return  dict(
                line=dict(
                color=line_color, width=0.5
            ),
            path =  make_q_bezier(list(b))+make_ribbon_arc(r[0], r[1])+make_q_bezier(list(c)[::-1])+make_ribbon_arc(l[1], l[0]),
            type='path',
            fillcolor=fill_color,
            layer='below'
        )

def make_self_rel(l, line_color, fill_color, radius):
    #radius is the radius of Bezier control point b_1
    b=control_pts([l[0], (l[0]+l[1])/2, l[1]], radius)
    return  dict(
                line=dict(
                color=line_color, width=0.5
            ),
            path=  make_q_bezier(b)+make_ribbon_arc(l[1], l[0]),
            type='path',
            fillcolor=fill_color,
            layer='below'
        )

def invPerm(perm):
    # function that returns the inverse of a permutation, perm
    inv = [0] * len(perm)
    for i, s in enumerate(perm):
        inv[s] = i
    return inv



In [25]:
radii_sribb=[0.4, 0.30, 0.35, 0.39, 0.12]# these value are set after a few trials 

In [26]:
shapes = []
ribbon_info=[]

for k in range(len(tech_terms_filters)):
    sigma=idx_sort[k]
    sigma_inv=invPerm(sigma)
    for j in range(k, len(tech_terms_filters)):
        if tech_terms_matrix[k][j]==0 and tech_terms_matrix[j][k]==0: continue
        eta=idx_sort[j]
        eta_inv=invPerm(eta)
        l=ribbon_ends[k][sigma_inv[j]]

        if j==k:
            shapes.append(make_self_rel(l, style['colors']['acc1'], ideo_colors[k], radius=radii_sribb[k]))
            z=0.9*np.exp(1j*(l[0]+l[1])/2)
            #the text below will be displayed when hovering the mouse over the ribbon
            text=tech_terms_labels[k]+' appears in '+ '{:d}'.format(tech_terms_matrix[k][k]),
            ribbon_info.append(
                go.Scatter(
                    x=[z.real],
                    y=[z.imag],
                    mode='markers',
                    marker=dict(size=0.5, color=ideo_colors[k]),
                    text=text,
                    hoverinfo='text',),)
        else:
            r=ribbon_ends[j][eta_inv[k]]
            zi=0.9*np.exp(1j*(l[0]+l[1])/2)
            zf=0.9*np.exp(1j*(r[0]+r[1])/2)
            texti=tech_terms_labels[k]+' appears with '+tech_terms_labels[j]+' {:d}'.format(tech_terms_matrix[k][j])+' times'
            textf=tech_terms_labels[j]+' appears with '+tech_terms_labels[k]+' {:d}'.format(tech_terms_matrix[j][k])+' times'
           
            ribbon_info.append(go.Scatter(x=[zi.real],
                                       y=[zi.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color='green'),
                                       text=texti,
                                       hoverinfo='text'
                                       )
                              ),
            ribbon_info.append(go.Scatter(x=[zf.real],
                                       y=[zf.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color='blue'),
                                       text=textf,
                                       hoverinfo='text'
                                       )
                              )
            r=(r[1], r[0])#IMPORTANT!!!  Reverse these arc ends because otherwise you get
                          # a twisted ribbon
            shapes.append(make_ribbon(l, r, 'rgb(175,175,175)' , ideo_colors[k]))

In [27]:
ideograms=[]
for k in range(len(ideo_ends)):
    z= make_ideogram_arc(1.1, ideo_ends[k])
    zi=make_ideogram_arc(1.0, ideo_ends[k])
    m=len(z)
    n=len(zi)
    ideograms.append(
        go.Scatter(
            x=z.real,
            y=z.imag,
            mode='lines',
            line=dict(color=ideo_colors[k], shape='spline', width=0.25),
            text=tech_terms_labels[k]+'<br>'+'{:d}'.format(row_sum[k]),
            hoverinfo='text',),)

    path='M '
    for s in range(m):
        path+=str(z.real[s])+', '+str(z.imag[s])+' L '

    Zi=np.array(zi.tolist()[::-1])

    for s in range(m):
        path+=str(Zi.real[s])+', '+str(Zi.imag[s])+' L '
    path+=str(z.real[0])+' ,'+str(z.imag[0])

    shapes.append(make_ideo_shape(path,'rgb(150,150,150)' , ideo_colors[k]))

data = go.Data(ideograms+ribbon_info)

/data/WORKSPACE/jpynb_Employment_Trends_Bulgaria/ve-emptrends/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [28]:
layout = go.Layout(
    paper_bgcolor=style['colors']['bg1'],            
    plot_bgcolor=style['colors']['bg1'],
    title = 'PoC Chord Diagram ‒ <br>Selected Techologies Commonly Occurring Together',
    titlefont=style['chart_fonts']['title'],
    font = style['chart_fonts']['text'],
    height = 525,
    margin = dict(
        l = 260,
        r = 260,
        t = 80,
        b = 10,
    ),
    showlegend=False,
    xaxis=dict(
        showline=False,
        zeroline=False,
        showgrid=False,
        showticklabels=False,
        title='',),
    yaxis=dict(
        showline=False,
        zeroline=False,
        showgrid=False,
        showticklabels=False,
        title='',),
    shapes=shapes,
    hovermode = 'closest',
    hoverdistance = 40,
)

In [29]:
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename = 'data_offers_tech_requirements_chord.html')

In [32]:
# Uncomment the line below to export an HTML version of the chart.
#plotly.offline.plot(fig, filename = 'data_offers_tech_requirements_chord.html')

'file:///data/WORKSPACE/jpynb_Employment_Trends_Bulgaria/workbooks/data_offers_tech_requirements_chord.html'

In [31]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)

### Resources:

- https://plot.ly/python/filled-chord-diagram/
- https://hci.stanford.edu/courses/cs448b/f11/lectures/CS448B-20111117-Text.pdf